# Introduction

The purpose of this notebook is to give an introduction to research with the CMB.  First, we'll use one of the major codes that are used to calculate cosmological models: not just the evolution of the scale factor over time, like we've already done in class, but also the power spectrum and many other things.  Second, we'll take a look at the data from the Planck satellite and compare it to our theoretical model.

This notebook is also the basis for Problem Set 4.

# CAMB
The Code for Anisotropies in the Microwave Background (CAMB) calculates the CMB power spectrum for a given set of input cosmological parameters.  It also has a more sophisticated cosmology solver (like we wrote in the 1st homework) that can do thinks like calculate distance as a function of redshift -- very handy!

We will only scratch the surface of what CAMB can do.  In addition to the Python interface, there is also a [web server](https://lambda.gsfc.nasa.gov/toolbox/camb_online.html) that will generate a CAMB model for you.  It is an easier way to see all of the possible input parameters.

We can specify what we want it to do and details about our cosmology, other assumptions through `CAMBparams`.

In [ ]:

from pylab import *
import camb
from camb import model, initialpower

## Creating a Cosmological Model

To begin with we need to create a new instance of a CAMB model.

In [ ]:
model = camb.CAMBparams()

Once we have created this new model, we can access some functions.  These three are required for generating a power spectrum.

In [ ]:
model.set_cosmology?

In [ ]:
model.InitPower.set_params?

In [ ]:
model.set_for_lmax?

First, let's create a cosmology with Planck 2018 parameters, using Table 1 from the [Planck 2018 paper VI.](https://arxiv.org/pdf/1807.06209.pdf); the table is on p. 15.  Note that running this next cell will produce a lot of text as output.  Something to consider: Why are each of these parameters relevant?  We haven't talked about all of them yet so it's okay if you don't know.

In [ ]:
model.set_cosmology(H0=67.37,
                   ombh2=0.02233, #baryon density
                   omch2=0.1198, #cold dark matter density
                   mnu=0.06, #single massive neutrino, mass in eV
                   omk=0, #curvature is flat
                   tau=0.054 #optical depth at reionization
                   )
model.InitPower.set_params(As=2.097e-9, ns=0.9652, r=0)
model.set_for_lmax(2000);

## Computing a Power Spectrum from the Model

Now we can run the code and generate some output, which includes the power spectrum.  You can also do things like get the current age of the Universe for the input cosmology.  See [this link](https://camb.readthedocs.io/en/latest/camb.html) for all of the top-level things you can do with CAMB.

In [ ]:
age = camb.get_age(model)
print("Age of the Universe is {0:6.3f} Gyr.".format(age))

And we can now plot the power spectrum.  Note that the output has much more information than we'll use so feel free to explore!

The different $C_{\ell}$ are always in the order TT, EE, BB, TE (with BB=0 for unlensed scalar results).

In [ ]:
results = camb.get_results(model)
?results.get_cmb_power_spectra

In [ ]:
powers =results.get_cmb_power_spectra(model, CMB_unit='muK')

Actually, what `results` gives us is the $D_{\ell} = \ell (\ell + 1) C_{\ell}/2 \pi$.  Recall from last time that $\sqrt{D_{\ell}} = \Delta T/T_0$.

In [ ]:
figure(1,figsize=(12,8))
Dl=powers['total']
plot(Dl[:,0], color='MediumOrchid', lw=2, label='H0=67');
xlim(2,2000) #first 2 ell-values are zero.
ylabel(r'$D^{TT}_{\ell}~[\mu K^2]$',fontsize=18)
xticks(fontsize=18)
yticks(fontsize=18)
legend(loc='upper right',frameon=False,fontsize=18)
xlabel(r'$\ell$',fontsize=18)
title("Figure 1",fontsize=18);

## Make some other plots!
Starting from our Figure 1, make a plot of $\sqrt{D_{\ell}}$, which is the temperature fluctuation in $\mu$K.


Use Figure 1 as a template to plot some of the polarization modes and cross-correlations (EE, TE, BB) as well.  When we asked for help on `results.get_cmb_power_spectra`, it says `where 0..3 index
are TT, EE, BB, TE` so when we made Figure 1, $D^{TT}_{\ell}$ we used `Dl[:,0]` (with zero for the second index of the `Dl` array.  To plot one of the polarization modes ($D^{EE}_{\ell}$) we'd used 1 instead of zero, and so on.


## Comparing to Planck Data

Does it look at all like the real thing?  Let's find out.

Here we'll plot our modelled power spectrum again along with the most recent data from Planck.  First we'll download the data file from NASA and then read it into memory.

In [ ]:
planck=genfromtxt('./data/TT_data_2020aug_csv_format.dat',skip_header=2,names=True,delimiter=',')

And make a comparison plot. The error bars on the high-$\ell$ points are tiny!

In [ ]:
figure(2,figsize=(12,8))
errorbar(x=planck['l_center'], y=planck['Power'], yerr=vstack((planck['Sigma_minus'], planck['Sigma_plus'])), fmt='+', color='MediumSeaGreen',label='Planck data')
#plot(Dl[:,0], color='MediumOrchid',label='CAMB model');
xlim(2,2000) #first 2 ell-values are zero.
ylabel(r'$D^{TT}_{\ell}~[\mu K^2]$',fontsize=18)
xticks(fontsize=18)
yticks(fontsize=18)
legend(loc='upper right',frameon=False,fontsize=18)
xlabel(r'$\ell$',fontsize=18)
title("Figure 2",fontsize=18);

Let's also make a log-plot so that we can see what's going on at low $\ell$.

In [ ]:
figure(3,figsize=(12,8))
errorbar(x=planck['l_center'], y=planck['Power'], yerr=vstack((planck['Sigma_minus'], planck['Sigma_plus'])), fmt='o', color='MediumSeaGreen', label='Planck data')
plot(Dl[:,0], color='MediumOrchid',label='CAMB model');
xlim(2,2000) #first 2 ell-values are zero.
ylabel(r'$D^{TT}_{\ell}~[\mu K^2]$',fontsize=18)
xticks(fontsize=18)
yticks(fontsize=18)
xscale('log')
legend(loc='upper left',frameon=False,fontsize=18)
xlabel(r'$\ell$',fontsize=18)
title("Figure 3",fontsize=18);

This is great!  It looks like some of the plots we looked at in class.  Of course, the good agreement between the data and the model is due to the fact that we're using the Planck 2018 parameters.

## Temperature Fluctuations from the Power Spectrum

We're accustomed to seeing the map of temperature fluctuations from a CMB experiment.  All the information required to compute such a map is stored in the power spectrum.  The following code allows us to make a small section of the full-sky map given the size of the image, number of pixels, and the input power spectrum $D_{\ell}$.

Run the code in the cell below; this contains two functions that will generate maps.  Run the following cell to create a plot.  Then run it again.  The map is a random realization of the power spectrum so each map will be different but look qualitatively the same.

You can fix the random number generator by setting the random seed using `np.random.seed(<some integer>)`.  This is necessary if you want to change the cosmological parameters but keep the map otherwise the same.

Try it for our standard model and your large variation.  How do the two maps compare?  Plot them side-by-side if possible.

I think the colormap that is typically used here is `Spectral_r` but you can try out a different one if you like.  Here is the [full list](https://matplotlib.org/stable/tutorials/colors/colormaps.html) of colormaps that you can choose from.

In [ ]:
#@title
def make_CMB_T_map(N,pix_size,ell,DlTT):
    "makes a realization of a simulated CMB sky map"

    # convert Dl to Cl
    ClTT = DlTT * 2 * np.pi / (ell*(ell+1))
    ClTT[0] = 0
    ClTT[1] = 0

    # make a 2d coordinate system
    ones = np.ones(N)
    inds  = (np.arange(N)+0.5 - N/2) /(N-1)
    X = np.outer(ones,inds)
    Y = np.transpose(X)
    R = np.sqrt(X**2 + Y**2)

    # now make a 2d CMB power spectrum
    ell_scale_factor = 2 * np.pi / (pix_size/60 * np.pi/180)
    ell2d = R * ell_scale_factor
    ClTT_expanded = np.zeros(int(ell2d.max())+1)
    ClTT_expanded[0:(ClTT.size)] = ClTT
    CLTT2d = ClTT_expanded[ell2d.astype(int)]

    # now make a realization of the CMB with the given power spectrum in fourier space
    ramdomn_array_for_T = np.fft.fft2(np.random.normal(0,1,(N,N)))
    FT_2d = np.sqrt(CLTT2d) * ramdomn_array_for_T
    CMB_T = np.fft.ifft2(np.fft.fftshift(FT_2d)) /(pix_size /60* np.pi/180)
    CMB_T = np.real(CMB_T)

    ## return the map
    return(CMB_T)

def Plot_CMB_Map(Map_to_Plot,c_min,c_max,X_width,Y_width):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    print("map mean (muK):",np.mean(Map_to_Plot),"map rms (muK):",np.std(Map_to_Plot))
    plt.figure(figsize=(10,10))
    im = plt.imshow(Map_to_Plot, interpolation='bilinear', origin='lower',cmap=cm.Spectral_r);
    im.set_clim(c_min,c_max)
    ax=plt.gca()
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    cbar = plt.colorbar(im, cax=cax)
    im.set_extent([0,X_width,0,Y_width])
    plt.show()
    return

In [ ]:
#here I am defining the array of l-values that correspond to the Dl
ell=arange(0,len(Dl[:,0]))

# this is the number of pixels in a linear dimension, should be a power of 2
N = pow(2,10)

pixel_size  = 0.5  # size of a pixel in arcminutes

## variables to set up the map plots
c_min = int(-400)  # minimum for color bar
c_max = int(400)   # maximum for color bar
X_width = N*pixel_size/60  # horizontal map width in degrees
Y_width = N*pixel_size/60 # vertical map width in degrees

#comment the next line to get different maps each time you run this cell
#np.random.seed(1111)

## make a CMB T map
CMB_T =make_CMB_T_map(N,pixel_size,ell,Dl[:,0]);
p = Plot_CMB_Map(CMB_T,c_min,c_max,X_width,Y_width);


Notice that to create the above plot we simply feed it $\ell$ and $D^{TT}_{\ell}$, along with some information about the number and size of the pixels.  That's because the power spectrum is really just the amplitudes of the temperature variations as a function of the (inverse) of the angular separations of those fluctuations.  The code that creates the map above just uses Fourier transforms to generate a random map given the power spectrum.

The preceding exercises are all about the theoretical side of the CMB.  The section that follows dives into the observational side.


# Healpy

There is a very powerful library called `healpy`, based on `HEALPix` ([more here](https://healpix.jpl.nasa.gov/pdf/intro.pdf)).  First we install the library and import it.

In [ ]:
import healpy as hp

Next we'll download the temperature maps from Planck Data Release 3.  These are large files so may take a minute or two.

In [ ]:
filename = './data/COM_CMB_IQU-commander_2048_R3.00_full.fits'
cmb_map = hp.read_map(filename)

This is the 'unmasked' map that includes Galactic emission.  To view it we'll use the Mollweide projection, as discussed previously.  The following produces a colormap that shows variations at the level of $10^{-3}$ K.

In [ ]:
hp.mollview(cmb_map, min=-1e-3, max=1e-3, title="CMB only temperature map", unit="K")

And this is the masked version where masks have been applied to remove the Galaxy and various point sources.

In [ ]:
path = './data/COM_Mask_CMB-common-Mask-Int_2048_R3.00.fits'
mask = hp.read_map(path)
map_masked = hp.ma(cmb_map)
map_masked.mask = np.logical_not(mask)
hp.mollview(map_masked, min=-1e-3, max=1e-3, title="CMB only temperature map", unit="K")

We can use the `healpy` library to calculate the power spectrum from these data.  Adapted from [here](https://zonca.dev/2021/02/compute-planck-spectra-healpy-anafast.html).

In [ ]:
?hp.anafast

The following cell will first make a power spectrum from the Planck data and then plot it together with the standard CAMB model we created above.  Note that the Healpy `anafast` routine returns $C_{\ell}$ so we need to convert to $D_{\ell}$ to compare with our CAMB model (or the reverse).

In [ ]:
lmax = 2000
test_cls_meas_frommap = hp.anafast(map_masked, lmax=lmax, use_pixel_weights=True)
ll = np.arange(lmax+1)
sky_fraction = len(map_masked.compressed()) / len(map_masked)
print(f"The map covers {sky_fraction:.1%} of the sky")
plt.style.use("seaborn-v0_8-poster")
k2muK = 1e6

plot(ell,Dl[:,0],color='MediumOrchid',label='CAMB model')
plot(ll, ll*(ll+1.)*test_cls_meas_frommap*pow(k2muK,2)/2./pi / sky_fraction, '--', color='MediumSeaGreen', alpha=0.6, label='Planck 2018 Data')
xlabel(r'$\ell$')
ylabel(r'$D_\ell~[\mu K^2]$')
legend(loc='best');

This looks OK but not great.  What's missing?  The width of the 'beam' when the satellite is observing is about 5 arcminutes.  As a first order correction, we can divide the measurements by the width of the beam.

In [ ]:
import astropy.units as u
width = hp.gauss_beam((5*u.arcmin).to_value(u.radian), lmax=lmax)
plot(ell,Dl[:,0],color='MediumOrchid',label='CAMB model')

plot(ll, ll*(ll+1.)*test_cls_meas_frommap*k2muK**2/2./np.pi / sky_fraction / pow(width,2),
         color='MediumSeaGreen', alpha=0.6, label='Planck 2018 Data (beam corrected)')
xlabel(r'$\ell$')
ylabel(r'$D_\ell~[\mu K^2]$')
legend(loc='best');

This is much better but still far from perfect.  To do any better than this we would have to do a full and proper analysis.  The Planck data file that we used to make Figures 2 and 3 above takes into account all of the sky masks as well as the beam size in great detail (that paper was 92 pages long!), which is way beyond the scope of this class!

## The End